**About** : This notebook is used to validate models.

In [1]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src/

/workspace/kaggle_islr/src


## Initialization

### Imports

In [3]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
torch.cuda.get_device_name(0)

1.14.0a0+410ce96


'Tesla V100-SXM2-32GB-LS'

In [4]:
import os
import re
import sys
import glob
import json
import time
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)
pd.set_option('display.max_columns', 500)

In [5]:
from params import *

from data.preparation import *
from data.dataset import SignDataset

from utils.metrics import *
from utils.plots import *
from utils.logger import Config

from inference.main import kfold_inference_val

## Expes

In [6]:
# EXP_FOLDER = "../logs/2023-03-16/42/"
# EXP_FOLDER = "../logs/2023-03-25/6/"
# EXP_FOLDER = "../logs/2023-03-29/5/"
EXP_FOLDER = "../logs/2023-03-30/3/"
EXP_FOLDER = "../logs/2023-04-06/10/"
EXP_FOLDER = "../logs/2023-04-07/12/"
EXP_FOLDER = "../logs/2023-04-09/0/"

In [7]:
config = Config(json.load(open(EXP_FOLDER + "config.json", "r")))

In [8]:
df = prepare_data(DATA_PATH, config.processed_folder)

In [17]:
EXP_FOLDERS = [
#     "../logs/2023-04-12/0/",
#     "../logs/2023-04-11/29/",
    "../logs/2023-04-12/4/",
    "../logs/2023-04-12/6/",
]

pred_oof = np.mean([np.load(e + "pred_oof.npy") for e in EXP_FOLDERS], 0)
df['pred'] = pred_oof.argmax(-1)

score = accuracy(df['target'], pred_oof)
print(f"-> CV acc : {score:.4f}")

-> CV acc : 0.7189


In [27]:
EXP_FOLDERS = [
    "../logs/2023-04-12/2/",
    "../logs/2023-04-12/0/",
#     "../logs/2023-04-11/31/",
    "../logs/2023-04-11/29/",
#     "../logs/2023-04-11/28/",
#     "../logs/2023-04-11/26/",
]

pred_oof = np.mean([np.load(e + "pred_oof.npy") for e in EXP_FOLDERS], 0)
df['pred'] = pred_oof.argmax(-1)

score = accuracy(df['target'], pred_oof)
print(f"-> CV acc : {score:.4f}")

-> CV acc : 0.7191


In [11]:
if "fold" not in df.columns:
    folds = pd.read_csv(config.folds_file)
    df = df.merge(folds, how="left", on=["participant_id", "sequence_id"])

In [12]:
pred_oof = np.load(EXP_FOLDER + "pred_oof.npy")
df['pred'] = pred_oof.argmax(-1)

score = accuracy(df['target'], pred_oof)
print(f"-> CV acc : {score:.4f}")

-> CV acc : 0.7158


In [13]:
# df = df[df['participant_id'] != 29302].reset_index(drop=True)
# score = accuracy(df['target'], df['pred'])
# print(f"-> CV acc : {score:.4f}")

In [14]:
df['error'] = (df['target'] != df['pred'])

In [15]:
dfg = df.groupby('participant_id').agg(['mean', 'count', 'sum'])[['error']]

In [16]:
dfg.sort_values(('error',  'mean'))

error            
                    mean count   sum
participant_id                      
26734           0.138195  4841   669
61333           0.159796  4900   783
62590           0.181460  4563   828
22343           0.192431  4677   900
37779           0.193225  4782   924
2044            0.219335  4810  1055
55372           0.221094  4826  1067
53618           0.226375  4656  1054
37055           0.255809  4648  1189
27610           0.262924  4275  1124
16069           0.269183  4848  1305
32319           0.292657  4753  1391
28656           0.295639  4563  1349
25571           0.302717  3865  1170
18796           0.313250  3502  1097
36257           0.325980  4896  1596
49445           0.327899  4968  1629
34503           0.370957  4545  1686
30680           0.406830  3338  1358
4718            0.540154  3499  1890
29302           0.590216  4722  2787

In [17]:
dfg = df.groupby('sign').agg('mean')[['error']].sort_values('error', ascending=False).T
dfg

sign,there,beside,give,pen,go,awake,goose,snack,nap,after,mouth,puzzle,fast,glasswindow,vacuum,ride,jeans,lamp,wake,hear,into,person,pencil,all,kitty,puppy,bedroom,mitten,helicopter,child,finger,kiss,close,room,sleep,jacket,cereal,chair,hide,tongue,yucky,sticky,look,sleepy,night,fish,on,open,dog,carrot,refrigerator,every,owie,hesheit,cloud,stay,zipper,hate,cheek,listen,scissors,car,boat,wet,can,jump,loud,chin,quiet,underwear,book,clean,backyard,tooth,drawer,dryer,napkin,penny,will,pizza,lips,fall,elephant,sun,haveto,pajamas,noisy,bee,alligator,cat,stairs,tree,grandma,pool,feet,animal,yes,drop,touch,any,that,read,cut,before,say,pretend,story,gift,garbage,have,fine,hen,empty,dance,outside,grass,another,finish,think,balloon,find,hello,morning,shirt,duck,weus,down,same,ear,farm,smile,bed,white,bad,wait,old,arm,rain,hair,blue,face,bath,red,man,table,happy,closet,why,flag,giraffe,pretty,yellow,like,bye,toy,zebra,many,girl,better,green,thankyou,time,cry,later,hot,make,snow,milk,potty,toothbrush,brother,blow,mouse,mom,talk,dry,not,hat,first,please,store,yourself,cowboy,sad,dirty,mad,up,minemy,shower,pig,thirsty,see,fireman,shoe,tomorrow,head,taste,police,TV,now,apple,grandpa,no,who,because,wolf,nuts,dad,yesterday,where,orange,frenchfries,moon,eye,boy,drink,for,food,hungry,home,aunt,sick,donkey,lion,cute,black,radio,stuck,icecream,if,high,bug,doll,cow,callonphone,frog,bird,chocolate,nose,clown,brown,airplane,tiger,owl,horse,gum,uncle,flower,water,shhh
error,0.705234,0.687097,0.65896,0.614035,0.608247,0.59601,0.586294,0.559055,0.544757,0.533141,0.531969,0.523392,0.522099,0.515385,0.504886,0.495677,0.494764,0.494595,0.488778,0.48642,0.483516,0.480769,0.464286,0.463731,0.460154,0.459384,0.458763,0.456233,0.453826,0.451149,0.448549,0.444444,0.441176,0.44086,0.431579,0.428571,0.427441,0.42663,0.424242,0.42246,0.422164,0.421203,0.42029,0.419355,0.416667,0.415789,0.415761,0.409605,0.407895,0.404145,0.402204,0.399457,0.39726,0.392157,0.391534,0.390957,0.384615,0.377095,0.373333,0.368675,0.368564,0.368272,0.364865,0.363874,0.363388,0.360313,0.359897,0.358382,0.357542,0.356568,0.355978,0.35493,0.354839,0.351145,0.348525,0.346883,0.346734,0.346154,0.344173,0.343434,0.343284,0.340314,0.335079,0.33419,0.33046,0.329049,0.32687,0.326582,0.325641,0.325,0.324607,0.324607,0.324607,0.322314,0.319892,0.318966,0.318653,0.317416,0.317204,0.315493,0.314961,0.314917,0.314363,0.30791,0.30504,0.30198,0.300813,0.300254,0.299191,0.296736,0.294444,0.29443,0.290323,0.288462,0.284974,0.284974,0.283784,0.280928,0.280702,0.279898,0.279597,0.274359,0.274194,0.272727,0.271605,0.269542,0.269113,0.268421,0.264781,0.264484,0.263298,0.260623,0.259459,0.257979,0.257225,0.257069,0.254144,0.251969,0.251948,0.251381,0.251351,0.250712,0.25066,0.247449,0.246291,0.240106,0.239186,0.239011,0.235751,0.235602,0.234694,0.234375,0.234043,0.233831,0.232804,0.232,0.230769,0.230159,0.228723,0.227749,0.22409,0.22314,0.223077,0.222812,0.221932,0.221106,0.217617,0.217507,0.216524,0.211443,0.210127,0.209719,0.208333,0.207595,0.203046,0.202046,0.201592,0.19883,0.198492,0.197917,0.195187,0.194805,0.190476,0.189258,0.189041,0.188776,0.187817,0.18617,0.185484,0.184783,0.180556,0.177835,0.176617,0.175824,0.17268,0.170984,0.168367,0.167526,0.166234,0.162602,0.161804,0.160207,0.159459,0.158809,0.156962,0.154639,0.154229,0.153439,0.153266,0.153226,0.151282,0.145833,0.142105,0.141772,0.141361,0.14,0.135417,0.135204,0.135065,0.134177,0.132992,0.128947,0.126829,0.125,0.123037,0.118557,0.116279,0.115607,0.115578,0.114754,0.114667,0.111111,0.1075,0.106436,0.101299,0.10101,0.09901,0.098446,0.096859,0.095361,0.094293,0.094148,0.093909,0.093434,0.089514,0.076923,0.076543,0.068182,0.065104,0.051095


In [18]:
classes = json.load(open(DATA_PATH + "sign_to_prediction_index_map.json", "r"))
classes = list(classes.keys())

In [19]:
cm = confusion_matrix(df['target'], df['pred'], normalize=None)

In [20]:
for i in tqdm(range(len(classes))):  # gt
    for j in range(len(classes)):
        n = cm[i, j]
        if n > 50 and i != j:
            s = f"{classes[i]} predicted as {classes[j]} :".ljust(32)
            print(f"{s} {n} / {cm[i].sum()}")

100%|██████████| 250/250 [00:00<00:00, 4621.07it/s]

awake predicted as wake :        165 / 401
duck predicted as goose :        60 / 405
give predicted as gift :         78 / 346
glasswindow predicted as tooth : 85 / 390
goose predicted as duck :        113 / 394
hear predicted as ear :          69 / 405
hear predicted as listen :       59 / 405
kiss predicted as home :         54 / 396
kitty predicted as cat :         70 / 389
lips predicted as mouth :        66 / 402
listen predicted as hear :       80 / 415
mouth predicted as lips :        124 / 391
nap predicted as sleep :         84 / 391
pen predicted as pencil :        118 / 399
pencil predicted as pen :        90 / 392
scissors predicted as cut :      61 / 369
sleepy predicted as sleep :      73 / 403
stay predicted as that :         58 / 376
that predicted as stay :         67 / 381
wake predicted as awake :        133 / 401


In [21]:
# plt.figure(figsize=(50, 50))
# plot_confusion_matrix(df['pred'], df['target'], display_labels=None)

### Explo

In [22]:
# path_to_glove_file = "../input/glove/glove.6B.50d.txt"

# embeddings_index = {}
# with open(path_to_glove_file, "rb") as f:
#     for line in tqdm(f):
#         word, coefs = line.split(maxsplit=1)
#         coefs = np.fromstring(coefs, "f", sep=" ")
#         embeddings_index[word.decode("utf-8")] = coefs

# print("Found %s word vectors." % len(embeddings_index))

In [23]:
# import spacy
# nlp = spacy.load('en_core_web_md')

# rep = {
#     "callonphone": 'phonecall',
#     "frenchfries": 'fries',
#     "glasswindow": 'window',
#     "hesheit": 'he',
#     "minemy": 'my',
#     "weus": 'we',
#     "haveto": "have",
#     "owie": "bruise",
# }

# embed = []
# for c in tqdm(classes):
#     c = rep.get(c, c)
# #     vec = nlp(c).vector
#     vec = embeddings_index[c.lower()]
#     embed.append(vec)
# embed = np.array(embed)

# # np.save('../output/embed.npy', embed)

In [24]:
# dists = np.sqrt((embed[:, None] - embed[None]) ** 2).mean(-1)

# # dists = (embed[:, None] * embed[None]).sum(-1) / ((embed[None] ** 2).sum(-1) * (embed[:, None] ** 2).sum(-1))

In [25]:
# for i in tqdm(range(len(classes))):  # gt
#     order = np.argsort(dists[i])
#     print(np.array(classes)[order[:5]])
# #     break

### Soup

In [26]:
EXP_FOLDER = "../logs/2023-04-06/10/"  # 0.7134
EXP_FOLDER = "../logs/2023-04-06/22/"  # 0.7126
EXP_FOLDER = "../logs/2023-04-06/25/"

In [27]:
config = Config(json.load(open(EXP_FOLDER + "config.json", "r")))
df = prepare_data(DATA_PATH, config.processed_folder)

if "fold" not in df.columns:
    folds = pd.read_csv(config.folds_file)
    df = df.merge(folds, how="left", on=["participant_id", "sequence_id"])
    
try:
    pred_oof = np.load(EXP_FOLDER + "pred_oof.npy")
    df['pred'] = pred_oof.argmax(-1)

    score = accuracy(df['target'], pred_oof)
    print(f"-> CV acc : {score:.4f}")
except:
    df = df[df['fold'] == 0].reset_index(drop=True)
    
    pred_oof = np.load(EXP_FOLDER + "pred_val_0.npy")
    df['pred'] = pred_oof.argmax(-1)

    score = accuracy(df['target'], pred_oof)
    print(f"-> Fold 0 acc : {score:.4f}")

-> CV acc : 0.7135


In [28]:
pred_oof = kfold_inference_val(
    df,
    EXP_FOLDER,
    debug=False,
    save=False,
    use_fp16=True,
)


- Fold 1

 -> Loading encoder weights from ../logs/2023-04-06/25/mlp_bert_3_0.pt



100%|██████████| 22/22 [00:38<00:00,  1.75s/it]



 -> Accuracy : 0.6859

- Fold 2

 -> Loading encoder weights from ../logs/2023-04-06/25/mlp_bert_3_1.pt



100%|██████████| 23/23 [00:37<00:00,  1.63s/it]



 -> Accuracy : 0.7400

- Fold 3

 -> Loading encoder weights from ../logs/2023-04-06/25/mlp_bert_3_2.pt



100%|██████████| 28/28 [00:51<00:00,  1.84s/it]



 -> Accuracy : 0.7076

- Fold 4

 -> Loading encoder weights from ../logs/2023-04-06/25/mlp_bert_3_3.pt



100%|██████████| 21/21 [00:45<00:00,  2.18s/it]



 -> Accuracy : 0.7232


 -> CV Accuracy : 0.7138


In [29]:
# pred = (pred_oof + np.load(EXP_FOLDER + "pred_val_0.npy")) / 2
# accuracy(df['target'], pred)

# df_val['pred_flip'] = pred.argmax(-1)
# df_val['error_flip'] = (df_val['pred_flip'] != df_val['target'])

# dfg = df_val.groupby('sign')[["error", "error_flip"]].mean()
# dfg["delta"] = dfg["error"] - dfg["error_flip"]

# dfg.sort_values('delta').T

# [classes.index(c) for c in dfg[dfg['delta'] < 0.01].index]

# [classes.index(c) for c in dfg[dfg['delta'] < -0.05].index]

## Inf train

In [30]:
EXP_FOLDER = "../logs/2023-04-11/27/"

In [31]:
config = Config(json.load(open(EXP_FOLDER + "config.json", "r")))
df = prepare_data(DATA_PATH, config.processed_folder)

if "fold" not in df.columns:
    folds = pd.read_csv(config.folds_file)
    df = df.merge(folds, how="left", on=["participant_id", "sequence_id"])

In [33]:
pred_oof = kfold_inference_val(
    df,
    EXP_FOLDER,
    debug=False,
    save=False,
    use_fp16=True,
    train=True
)


- Fold 1

 -> Loading encoder weights from ../logs/2023-04-11/27/mlp_bert_3_0.pt



100%|██████████| 71/71 [00:39<00:00,  1.78it/s]



 -> Accuracy : 0.8518

- Fold 2

 -> Loading encoder weights from ../logs/2023-04-11/27/mlp_bert_3_1.pt



100%|██████████| 71/71 [01:11<00:00,  1.01s/it]



 -> Accuracy : 0.8315

- Fold 3

 -> Loading encoder weights from ../logs/2023-04-11/27/mlp_bert_3_2.pt



100%|██████████| 65/65 [01:16<00:00,  1.18s/it]



 -> Accuracy : 0.8329

- Fold 4

 -> Loading encoder weights from ../logs/2023-04-11/27/mlp_bert_3_3.pt



100%|██████████| 72/72 [01:37<00:00,  1.36s/it]



 -> Accuracy : 0.8403


 -> CV Accuracy : 0.6305


In [34]:
np.save(EXP_FOLDER + "pred_oof_train.npy", pred_oof)

#### Checks

In [40]:
df['pred_0'] = pred_oof[0].argmax(-1)
df_val = df[df['fold'] != 0].reset_index(drop=True)

In [44]:
df_err = df_val[(df_val['target'] != df_val['pred_0'])].reset_index(drop=True)

In [48]:
dataset = SignDataset(df_err, max_len=None, train=False)

In [ ]:
for i in tqdm(range(len(dataset))):
#     i = 92284
    data = dataset[i]
    
#     for k in data.keys():
#         print(k, data[k].size())
    
    print(df_err['sequence_id'][i], "- pred :", classes[df_err['pred_0'][i]], " - truth :", df_err['sign'][i])
    plot_sample_with_edges(data, n_frames=4, figsize=(10, 10), show_text=False)

    if i > 10:
        break

Done ! 